# Try Langchain with Baidu

## 1. Integrate Baidu API to Langchain

The following is the effort to integrate Baidu API to Langchain.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.prompts import PromptTemplate

synopsis_prompt = PromptTemplate.from_template("""你是一位编剧。鉴于电影的标题，你的任务是为该标题撰写一个电影剧情的主干。

标题：{title}
编剧：以下是上述电影的主干："""
)

review_template =  """你是《豆瓣影评》的深度用户。鉴于该电影的故事主干，你的任务是为该电影撰写一篇宣传称赞的评论。

主干：
{synopsis}
《豆瓣影评》戏剧评论家对上述戏剧的评论：:"""

review_prompt = PromptTemplate.from_template(review_template)

title="众里寻他千百度"

## LCEL

In [4]:
from langchain.chat_models import QianfanChatEndpoint 
from langchain.schema import StrOutputParser

llm = QianfanChatEndpoint()
chain = (
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)
chain.invoke({"title": title})

'《众里寻他千百度》是一部值得一看的电影，它不仅拥有引人入胜的故事情节，还展现了人类内心的丰富情感和人性中的光明面。这部电影的情节紧凑，让人无法停下脚步，不断地想继续看下去。\n\n电影的主干故事非常精彩，它构建了一个充满神秘色彩和情感力量的世界——“浮生城”。这个城市因为一种神秘的力量而繁荣昌盛，这种力量就是人类的心灵之光。电影的主角李千寻，是一位有着不凡梦想和勇气的青年，他拥有一双看穿表象的眼睛，为了解开“浮生城”背后的真相，一直探寻着城市的秘密。他的寻梦之旅并不是一帆风顺，但他总能在绝望之际找到希望之光，让人们重拾内心之光的力量。\n\n电影中的人物形象鲜明，他们都有自己的故事和情感，这些故事和情感通过人物的交流与冲突展现出来，描绘出人的心灵世界和这座城市共同编织出的奇迹。这些人物的情感冲突和故事发展让电影更加吸引人，也增加了电影的深度和内涵。\n\n此外，电影中的笑点和紧张的冲突也为故事增色不少。这些笑点和冲突让电影更加有趣，同时也展示了人类情感的复杂性和多样性。但是，尽管这些笑点和冲突丰富了电影的内容，主线剧情仍是围绕着李千寻对“浮生城”的探索，以及他如何通过自己的努力和智慧去解开这个城市的秘密。\n\n总的来说，《众里寻他千百度》是一部非常出色的电影，它不仅具有引人入胜的故事情节，还展现了人类内心的丰富情感和人性中的光明面。我强烈推荐这部电影给所有喜欢看故事片的人。这部电影将会让你沉浸在它的神秘世界中，感受到人性的光辉和无尽的探索精神。'

通过`RunnablePassthrough.assign`我们可以将将之前的输入字段进行透传，并分配我们本次的输出和字段名

In [5]:
from langchain.schema.runnable import RunnablePassthrough

synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = {"synopsis": synopsis_chain} | RunnablePassthrough.assign(custom_review_field=review_chain)
chain.invoke({"title": title})

{'synopsis': '电影《众里寻他千百度》的剧情主干：\n\n电影《众里寻他千百度》讲述的是一位年轻的女主角赵心怡在经历人生挫折后，陷入自我怀疑和困惑。在这个过程中，她突然失去了一部分记忆，但却一直寻找着自己的真实身份和过去。她走遍世界各地，追寻着自己的故事，也在过程中重新认识了自我和人生。\n\n剧情展开：\n\n一、前半部分：失忆与自我怀疑\n赵心怡在一次意外中失去了记忆，她的世界变得混乱不堪。她开始怀疑自己的身份和过去，感到迷茫和无助。在这个过程中，她遇到了各种人，包括她的朋友、家人和陌生人，他们帮助她寻找答案，但同时也让她更加困惑。\n\n二、中半部分：寻找真相\n赵心怡开始在世界各地寻找自己的过去和真相。她在古老的城市中寻找遗迹，试图揭开过去的谜团。在旅程中，她逐渐认识到了自我，同时也对生活有了更深的理解和认识。\n\n三、后半部分：战胜自我，回归现实\n赵心怡最终战胜了自我怀疑和困惑，找到了自己的真实身份和过去。在这个过程中，她学会了如何面对自己的弱点和缺陷，也学会了如何去爱自己和他人。最后，她回到了现实世界，重新开始了自己的生活。\n\n电影的主题是寻找自我和人生意义，通过赵心怡的旅程，观众可以从中获得启示和鼓励，学会如何面对生活的挫折和困惑，重新找回自我和人生方向。同时，这部电影也将探讨现代社会中人们的迷茫和困惑，以及寻找真实的自我和人生的意义的重要性。',
 'custom_review_field': '这篇《豆瓣影评》非常生动地描述了电影《众里寻他千百度》的主干剧情，深入剖析了电影的主题和意义。\n\n首先，该电影以一位年轻女性失去记忆和自我怀疑为起点，引发了观众对自我和人生的深度思考。这个主题在当今社会中具有很强的现实意义，它鼓励我们面对自己的弱点和缺陷，重新找回自我和人生方向。\n\n其次，电影中的赵心怡在世界各地寻找自己的过去和真相，她的旅程充满了未知和挑战，但同时也充满了希望和启示。这个情节设计非常巧妙，它让观众跟随赵心怡一起经历人生，感受其中的酸甜苦辣。\n\n此外，这部电影探讨了现代社会中人们的迷茫和困惑，它让观众反思我们如何在寻找真实自我的过程中找到人生的意义。电影中的情节和人物角色非常饱满，故事情节跌宕起伏，让观众一直保持高度的关注。\n\n总的来说，这部电影是一部深度和内涵丰富的佳作。它的成功在于剧情的设计和人物形象的

## SimpleSequentialChain

In [6]:
from langchain.chains import LLMChain
from langchain.chat_models import QianfanChatEndpoint 
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = QianfanChatEndpoint(model="ERNIE-Bot", temperature=0.5)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt)

In [7]:
review_chain = LLMChain(llm=llm, prompt=review_prompt)

In [8]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

review = overall_chain.run(title)

/home/za/gpts/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
电影《众里寻他千百度》剧情主干：

1. 引子：主人公，一位年轻有为的企业家李昊，在一次偶然的机会中，遇到了一位神秘女子雨晴。两人一见钟情，然而雨晴却突然消失，留下李昊独自在人群中寻找。
2. 追寻：李昊决定寻找雨晴，他通过各种途径，包括调查、询问、网络搜索等，历经千辛万苦，却始终未能找到雨晴的踪迹。在这个过程中，他不仅深入体验了人间的冷暖，还逐渐理解了什么是真正的爱情和人生。
3. 转折：在一次偶然的机会中，李昊发现了一个与雨晴有关的线索，这个线索引领他来到一个偏远的山村。在这里，他结识了一位老者，老者告诉他雨晴曾是他的邻居，因为一场意外而离开了这个世界。
4. 结局：李昊虽然无法找到雨晴，但他从老者那里领悟到了生命的真谛和爱情的意义。他决定放下过去，珍惜当下，勇敢地面对未来。影片以李昊回到城市后，用更加积极的态度去面对生活和工作作为结尾，传递出积极向上的人生观和价值观。
《众里寻他千百度》是一部感人至深的爱情故事，它以独特的视角展现了人生的意义和价值。影片通过李昊的寻找之旅，展现了人间的冷暖和爱情的真谛。在这个过程中，观众不仅会被影片的情节所吸引，更会被影片所传递的人生观和价值观所感动。影片的演员表现出色，导演的拍摄手法独特，整部影片给人留下了深刻的印象。如果你在寻找一部感人至深的爱情故事，那么《众里寻他千百度》绝对值得一看。

> Finished chain.


In [9]:
print(review)

《众里寻他千百度》是一部感人至深的爱情故事，它以独特的视角展现了人生的意义和价值。影片通过李昊的寻找之旅，展现了人间的冷暖和爱情的真谛。在这个过程中，观众不仅会被影片的情节所吸引，更会被影片所传递的人生观和价值观所感动。影片的演员表现出色，导演的拍摄手法独特，整部影片给人留下了深刻的印象。如果你在寻找一部感人至深的爱情故事，那么《众里寻他千百度》绝对值得一看。


## SequentialChain

In [10]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
synopsis_template = """"你是一位编剧。鉴于电影的标题和背景，你的任务是为该标题撰写一个电影剧情的主干。

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)
synopsis_chain = LLMChain(
    llm=llm, prompt=synopsis_prompt_template, output_key="synopsis"
)

In [11]:
# This is an LLMChain to write a review of a play given a synopsis.
# template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

# Play Synopsis:
# {synopsis}
# Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=review_template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

era = "21世纪20年代的广东深圳"
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)


overall_chain({"title": title, "era": era})

/home/za/gpts/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'title': '众里寻他千百度',
 'era': '21世纪20年代的广东深圳',
 'synopsis': 'Title: 众里寻他千百度\n\nEra: 21世纪20年代的广东深圳\n\nPlaywright:\n\n《众里寻他千百度》是一部描绘现代都市爱情的电影。故事发生在中国南方的大都市深圳，这里充满了机遇与挑战，是众多年轻人追逐梦想的地方。\n\n男主角李浩是一个来自东北的年轻人，怀揣着对未来的憧憬来到深圳。他梦想着在这座城市里找到自己的定位，实现自己的理想。在努力打拼的过程中，李浩结识了女主角林雨菲，一个美丽、独立、善良的女孩。两人在共同奋斗的日子里，渐渐产生了感情。\n\n然而，生活并非一帆风顺。李浩的家庭出现了经济危机，他不得不承担起更多的责任。与此同时，林雨菲也面临着职业上的挑战和选择。两人的感情在现实的压力下开始出现裂痕。\n\n在一次偶然的机会下，李浩发现了一份高薪的工作机会，但这份工作需要他离开深圳，前往遥远的西部城市。面对这个选择，李浩犹豫不决。他知道，如果选择这份工作，他和林雨菲的关系可能会因此而结束。但如果放弃这个机会，他又担心无法承担起家庭的重担。\n\n在经过深思熟虑后，李浩决定接受这份工作。他告诉林雨菲自己的决定，并希望她能理解。林雨菲虽然心中痛苦，但她仍然支持李浩的决定，希望他能够实现自己的理想。\n\n在离别的时刻，两人相约在深圳的街头。他们手牵手漫步在繁华的都市中，回忆着过去的点滴。他们深知，即使身处不同的城市，他们的心依然紧紧相连。\n\n电影以两人在夜幕降临时的告别为结尾，寓意着他们虽然面临着重重困难和挑战，但依然坚定地追寻着自己的梦想和爱情。',
 'review': '《众里寻他千百度》是一部深入人心的爱情故事，它细腻地描绘了现代都市中年轻人的生活、梦想和爱情。影片中的角色形象鲜明，情节紧凑，让人不禁为他们的遭遇和选择而动容。\n\n首先，影片的背景设定在充满活力的深圳，这座城市既是梦想的起点，也是爱情的温床。在这里，我们见证了男主角李浩如何从一名东北的小伙子，怀揣着对未来的憧憬，勇敢地踏上了追逐梦想的征程。他的坚持和努力，不仅让我们看到了年轻人对未来的渴望，也让我们感受到了他对爱情的执着。\n\n女主角林雨菲的出场，为这个故事增添了更多的色彩。她美丽、独立、善良，是李浩在深圳奋斗过程中最坚实的后盾。两人在共

## Use memory

In [12]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

template = """你是一家电影公司的社交媒体经理。鉴于电影的标题、设定的时代、日期、时间和地点，以及电影的主干和评论，你的任务是为该电影撰写一则社交媒体帖子。

以下是关于电影的时间和地点的一些背景信息：:
时间: {time}
地点: {location}

电影主干:
{synopsis}
电影评论：
{review}

社交媒体帖子:
"""

time = "12月初冬，早7点"
location = "南山后海深圳湾公园"
prompt_template = PromptTemplate(
    input_variables=["synopsis", "review", "time", "location"], template=template
)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(
        memories={"time": time, "location": location}
    ),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True,
)

overall_chain({"title": title, "era": era})



> Entering new SequentialChain chain...

> Finished chain.


{'title': '众里寻他千百度',
 'era': '21世纪20年代的广东深圳',
 'time': '12月初冬，早7点',
 'location': '南山后海深圳湾公园',
 'social_post_text': '《众里寻他千百度》——在繁华的深圳，探寻真挚的爱与幸福\n\n冬日的早晨，深圳湾公园的晨雾还未完全散去，一部深入人心的爱情故事《众里寻他千百度》即将上演。在21世纪20年代的深圳，艺术家李梁和企业家陈的命运再次交织。\n\n他们曾是大学时代的恋人，因梦想和人生道路而分离。李梁在深圳的艺术圈崭露头角，而陈在家乡创业，成为了一名成功的企业家。但在这座充满机遇与诱惑的城市中，他们都曾迷失，对真正的幸福感到迷茫。\n\n命运使他们再次相遇。尽管身份、地位有了巨大差异，但他们对彼此的感情依然如初。然而，名利和地位的诱惑，以及两人生活的巨大差异，使得他们难以找回曾经纯真的爱情。\n\n这部影片不仅是一个爱情故事，更是一次对人生选择和价值观的深度反思。在追求名利的过程中，我们是否也曾迷失自我？在经历了种种挫折后，我们是否还能找回内心的平静和真正的幸福？\n\n《众里寻他千百度》将带你一同探寻这些问题。在这个冬日的早晨，让我们在南山后海深圳湾公园一同感受这份真挚的爱与幸福。\n\n#众里寻他千百度 #深圳爱情故事 #冬日温暖'}